# Integration into frameworks

## Mesa

In [74]:
import pop2net as p2n
import mesa

In [75]:
class Agent(p2n.Actor, mesa.Agent):
    def say_hello(self):
        print("Hello I am an actor.")

In [ ]:
class Location(p2n.Location, mesa.Agent):
    def say_hello(self):
        print("Hello I am a location.")

In [ ]:
class Model(mesa.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # add pop2net environment object as model attribute
        self.env = p2n.Environment(model=self, framework="mesa")

        # add one actor to the environment
        self.env.add_actor(Agent(model=self))

        # add one location to the environment
        self.env.add_location(Location(model=self))

    def step(self):
        # Because env.actors and env.locations are AgentSets now,
        # we can use the Mesa Syntax to let the agents do something:
        self.env.actors.do("say_hello")
        self.env.locations.do("say_hello")

In [78]:
model = Model()

In [79]:
model.agents

In [80]:
list(model.agents)

[<__main__.Agent at 0x75daa3b964b0>, <__main__.Location at 0x75daa3b96a20>]

In [81]:
model.env.actors

In [82]:
model.step()

Hello I am an actor.
Hello I am a location.


In [ ]:
class Model(mesa.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # add pop2net environment object as model attribute
        self.env = p2n.Environment(model=self, framework="mesa")

        # add one actor to the environment
        self.env.add_actor(Agent(model=self))

        # add one location to the environment
        self.env.add_location(Location(model=self))

    def step(self):
        # Because env.actors and env.locations are AgentSets now,
        # we can use the Mesa Syntax to let the agents do something:
        self.env.actors.do("say_hello")
        self.env.locations.do("say_hello")

## AgentPy